<h2> Instalar Todas as Bibliotecas<h2>

In [1]:
pip install requests

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install kafka-python

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install cassandra-driver

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pymongo

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import requests
import json
from kafka import KafkaProducer
from kafka import KafkaConsumer
from pymongo import MongoClient
import pymongo
import json
import pandas as pd
from cassandra.cluster import Cluster
from cassandra.cqlengine import columns
from cassandra.cqlengine.models import Model
from cassandra.cqlengine import connection
from cassandra.cqlengine.management import sync_table, create_keyspace_simple
from datetime import datetime
from cassandra.auth import PlainTextAuthProvider
from email.mime.text import MIMEText
import smtplib
import time

<h2>Inicializar Kafka, acessar APIs e enviar dados para Kafka<h2>
<h3> API CoinMarketCap com cotação atual do Bitcoin<h3>
<h3> API alternative.me com Índice de Medo e Ganância<h3>
<h3> API Crypto Panic notícias e análise de sentimento<h3>

In [19]:

# Kafka setup
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092', 'localhost:9093', 'localhost:9094'],
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# =======================
# 1. CoinMarketCap - Cotação Bitcoin
# =======================
def fetch_crypto_price():
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': "ee788f7c-e289-4d2d-8f1f-59d0e00db8c3",
    }

    params = {
        "id": "1"  # Bitcoin
    }

    url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest"
    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            print("🪙 Crypto Price Data:")
            print(data)

            # enviar para Kafka
            producer.send('crypto-prices', value=data)
            producer.flush()
            print("✅ Enviado para o Kafka: crypto-prices")
        else:
            print("❌ Erro para conseguir cotação:", response.status_code, response.text)
    except Exception as e:
        print("❌ Exceção para conseguir cotação:", e)

# =======================
# 2. Índice de Medo e Ganância
# =======================
def get_fear_and_greed_index():
    url = "https://api.alternative.me/fng/"
    try:
        response = requests.get(url)
        data = response.json()
        index_value = data["data"][0]["value"]
        index_status = data["data"][0]["value_classification"]

        result = {
            "index": index_value,
            "status": index_status,
            "raw": data["data"][0]  # Optional: include raw data
        }

        print("📊 Índice Fear & Greed:")
        print(f"- Índice: {result['index']}")
        print(f"- Status: {result['status']}")

        # Send to Kafka
        producer.send('fear-greed-index', value=result)
        producer.flush()
        print("✅ Enviado para o Kafka: fear-greed-index")
        print(result)
    except Exception as e:
        print("❌ Erro em alternative.me:", e)

# =======================
# 3. CryptoPanic - Notícias para Análise de Sentimento
# =======================
def get_cryptopanic_news():
    api_key = "1a4c3fd0802d6e2796140908ed1f411df4a10414"  # Optional, if you have one
    url = "https://cryptopanic.com/api/v1/posts/"
    
    params = {
        "auth_token": api_key,       # Optional: you can remove this line if no key
        "currencies": "BTC",
        "public": "true"
    }

    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            news_items = data.get("results", [])
            
            print("📰 CryptoPanic News Headlines:")
            for item in news_items[:5]:  # Just show top 5 in console
                print(f"- {item['title']} ({item['published_at']})")

            # Send to Kafka
            producer.send('crypto-news', value=news_items)
            producer.flush()
            print("✅ Enviado para o Kafka: crypto-news")
        else:
            print("❌ Erro na API do CryptoPanic:", response.status_code, response.text)
    except Exception as e:
        print("❌ Exceção ao acessar CryptoPanic:", e)


# =======================
# Execução das funções
# =======================
if __name__ == "__main__":
    fetch_crypto_price()
    get_fear_and_greed_index()
    get_cryptopanic_news()


🪙 Crypto Price Data:
{'status': {'timestamp': '2025-04-20T23:58:57.129Z', 'error_code': 0, 'error_message': None, 'elapsed': 51, 'credit_count': 1, 'notice': None}, 'data': {'1': {'id': 1, 'name': 'Bitcoin', 'symbol': 'BTC', 'slug': 'bitcoin', 'num_market_pairs': 12042, 'date_added': '2010-07-13T00:00:00.000Z', 'tags': ['mineable', 'pow', 'sha-256', 'store-of-value', 'state-channel', 'coinbase-ventures-portfolio', 'three-arrows-capital-portfolio', 'polychain-capital-portfolio', 'binance-labs-portfolio', 'blockchain-capital-portfolio', 'boostvc-portfolio', 'cms-holdings-portfolio', 'dcg-portfolio', 'dragonfly-capital-portfolio', 'electric-capital-portfolio', 'fabric-ventures-portfolio', 'framework-ventures-portfolio', 'galaxy-digital-portfolio', 'huobi-capital-portfolio', 'alameda-research-portfolio', 'a16z-portfolio', '1confirmation-portfolio', 'winklevoss-capital-portfolio', 'usv-portfolio', 'placeholder-ventures-portfolio', 'pantera-capital-portfolio', 'multicoin-capital-portfolio', 

<h2> Criar um Consumidor para Kafka e enviar dados para armazenamento no MongoDB (Data Lake)<h2>

In [20]:


# Import pymongo
from pymongo import errors

# MongoDB setup
mongo_client = MongoClient("mongodb://localhost:27017/")
db = mongo_client["crypto_data"]

# Kafka consumer setup
consumer = KafkaConsumer(
    'crypto-prices', 'fear-greed-index', 'crypto-news',
    bootstrap_servers=['localhost:9092', 'localhost:9093', 'localhost:9094'],
    value_deserializer=lambda m: json.loads(m.decode('utf-8')),
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='crypto-group6'
)

print("📡 Ouvindo Mensagens do Kafka...")

print("📚 Tópicos disponíveis no Kafka:", consumer.topics())

from datetime import datetime, timedelta

timeout = timedelta(seconds=30)
end_time = datetime.now() + timeout

while datetime.now() < end_time:
    message_pack = consumer.poll(timeout_ms=1000)
    print("🔄 Verificando novas mensagens...")
    print("📦 Pacote de mensagens recebido:", message_pack)
    for topic_partition, messages in message_pack.items():
        for message in messages:
            print("🔄 Mensagem recebida!")
            topic = message.topic
            value = message.value

            print(f"📥 Mensagem recebida do tópico '{topic}': {value}")

            if topic == 'crypto-prices':
                try:
                    db.crypto_prices.insert_one(value)
                    print("✅ Inserido no MongoDB: crypto_prices")
                except pymongo.errors.PyMongoError as e:
                    print(f"❌ Erro ao inserir no MongoDB (crypto_prices): {e}")

            elif topic == 'fear-greed-index':
                try:
                    db.fear_greed_index.insert_one(value)
                    print("✅ Inserido no MongoDB: fear_greed_index")
                except pymongo.errors.PyMongoError as e:
                    print(f"❌ Erro ao inserir no MongoDB (fear_greed-index): {e}")
            elif topic == 'crypto-news':
                try:
                    if isinstance(value, list):
                        db.crypto_news.insert_many(value)
                        print(f"✅ {len(value)} notícias inseridas no MongoDB: crypto_news")
                    else:
                        db.crypto_news.insert_one(value)
                        print("✅ 1 notícia inserida no MongoDB: crypto_news")
                except pymongo.errors.PyMongoError as e:
                    print(f"❌ Erro ao inserir no MongoDB (crypto-news): {e}")

📡 Ouvindo Mensagens do Kafka...
📚 Tópicos disponíveis no Kafka: {'fear-greed-index', 'crypto-prices', 'crypto-news'}
🔄 Verificando novas mensagens...
📦 Pacote de mensagens recebido: {}
🔄 Verificando novas mensagens...
📦 Pacote de mensagens recebido: {}
🔄 Verificando novas mensagens...
📦 Pacote de mensagens recebido: {}
🔄 Verificando novas mensagens...
📦 Pacote de mensagens recebido: {TopicPartition(topic='fear-greed-index', partition=0): [ConsumerRecord(topic='fear-greed-index', partition=0, leader_epoch=6, offset=7, timestamp=1745193537826, timestamp_type=0, key=None, value={'index': '37', 'status': 'Fear', 'raw': {'value': '37', 'value_classification': 'Fear', 'timestamp': '1745107200', 'time_until_update': '63'}}, headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=143, serialized_header_size=-1)], TopicPartition(topic='crypto-news', partition=0): [ConsumerRecord(topic='crypto-news', partition=0, leader_epoch=6, offset=7, timestamp=1745193538376, timestamp_type=

<h2> Transformação de dados em Informação que pode ser Utilizada</h2>
<p>Acessar e Transformar dados de todas as fontes enviadas para o MongoDB</p>
<p> Armazenar dados estruturados em Cassandra (Data Warehouse)</p>

In [21]:
# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["crypto_data"]

# Collections
price_collection = db["crypto_prices"]
fear_greed_collection = db["fear_greed_index"]
news_collection = db["crypto_news"]


# Extrair dados relevantes da cotação
price_records = []
for doc in price_collection.find():
    try:
        btc_data = doc["data"]["1"]
        quote = btc_data["quote"]["USD"]
        price_record = {
            "timestamp": pd.to_datetime(doc["status"]["timestamp"]),
            "price": quote["price"],
            "percent_change_1h": quote["percent_change_1h"],
            "percent_change_24h": quote["percent_change_24h"],
            "percent_change_30d": quote["percent_change_30d"]
        }
        price_records.append(price_record)
    except KeyError:
        continue

# --- Extrair dados relevantes do Fear and Greed Index ---
fear_greed_records = []
for doc in fear_greed_collection.find():
    try:
        timestamp_raw = doc["raw"]["timestamp"]
        index_timestamp = pd.to_datetime(int(timestamp_raw), unit='s')  # Converter para inteiro antes de pd.to_datetime
        value = int(doc["raw"]["value"])
        value_classification = doc["raw"]["value_classification"]
        fear_greed_record = {
            "timestamp": index_timestamp,
            "value": value,
            "status": value_classification
        }
        fear_greed_records.append(fear_greed_record)
    except KeyError as e:
        print(f"Erro ao processar documento Fear and Greed: KeyError - {e}")
        continue
    except TypeError as e:
        print(f"Erro ao processar documento Fear and Greed: TypeError - {e}")
        continue

# --- Extrair dados relevantes da news_collection ---
news_records = []
for doc in news_collection.find():
    try:
        title = doc["title"]
        url = doc["url"]
        published_at_str = doc["published_at"]
        # Convert '2025-04-19T18:30:00Z' to datetime object
        published_at = datetime.strptime(published_at_str, '%Y-%m-%dT%H:%M:%SZ')
        news_record = {
            "published_at": published_at,
            "title": title,
            "url": url
        }
        news_records.append(news_record)
    except KeyError as e:
        print(f"Erro ao processar documento de notícia: KeyError - {e}")
        continue
    except ValueError as e:
        print(f"Erro ao processar documento de notícia (data): ValueError - {e}")
        continue

# Criar DataFrame para Cotação
btc_prices_df = pd.DataFrame(price_records)
btc_prices_df.sort_values("timestamp", inplace=True)
btc_prices_df.reset_index(drop=True, inplace=True)
display(btc_prices_df.head())

# Criar DataFrame para Fear and Greed Index
fear_greed_df = pd.DataFrame(fear_greed_records)
fear_greed_df.sort_values("timestamp", inplace=True)
fear_greed_df.reset_index(drop=True, inplace=True)
display(fear_greed_df.head())

# Criar DataFrame para as notícias
news_df = pd.DataFrame(news_records)
news_df.sort_values("published_at", inplace=True)
news_df.reset_index(drop=True, inplace=True)
display(news_df.head())

# --- Conectar com Cassandra ---
cluster = None
try:
    cluster = Cluster(['127.0.0.1'])
    cassandra_session = cluster.connect()
    print("Conectado Cassandra!")

    # Teste de conexão
    row = cassandra_session.execute("SELECT now() FROM system.local").one()
    if row:
        print(f"Sucesso para executar consulta: {row[0]}")

    # Inicializar o cqlengine
    connection.setup(hosts=['127.0.0.1'], default_keyspace='crypto_keyspace')

    # Definir o nome do keyspace globalmente
    KEYSPACE = 'crypto_keyspace'

    # Definir modelo e criar keyspace
    try:
        cassandra_session.execute(f"""
            CREATE KEYSPACE IF NOT EXISTS {KEYSPACE}
            WITH REPLICATION = {{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }}
        """)
        print(f"Keyspace '{KEYSPACE}' criado ou já existe.")
    except Exception as e:
        print(f"Erro criando keyspace '{KEYSPACE}': {e}")
    cassandra_session.set_keyspace(KEYSPACE)
    print(f"Keyspace '{KEYSPACE}' definido.")

    class BTCPrices(Model):
            __keyspace__ = KEYSPACE
            __tablename__ = 'btc_prices'
            partition_key = columns.Text(primary_key=True)
            price_timestamp = columns.DateTime(primary_key=True, clustering_order='ASC')
            price = columns.Float()
            percent_change_1h = columns.Float()
            percent_change_24h = columns.Float()
            percent_change_30d = columns.Float()

    # Definir modelo para Fear and Greed Index
    class FearGreedIndex(Model):
        __keyspace__ = KEYSPACE
        __tablename__ = 'fear_greed_index'
        index_timestamp = columns.DateTime(primary_key=True)
        value = columns.Integer()
        status = columns.Text()

    # Definir modelo para as notícias
    class News(Model):
        __keyspace__ = KEYSPACE
        __tablename__ = 'news'
        published_at = columns.DateTime(primary_key=True)
        title = columns.Text()
        url = columns.Text()

    sync_table(News)
    print(f"Tabela Cassandra Notícias '{News.__tablename__}' sincronizada.")

    sync_table(FearGreedIndex)
    print(f"Tabela Cassandra Índice de Medo e Ganância '{FearGreedIndex.__tablename__}' sincronizada.")

    sync_table(BTCPrices)
    print(f"Tabela Cassandra Preços BTC '{BTCPrices.__tablename__}' sincronizada.")

    # --- Armazenar dados das notícias no Cassandra ---
    print("Armazenando dados das notícias no Cassandra...")
    for index, row in news_df.iterrows():
        news_record_cassandra = News(
            published_at=row['published_at'].to_pydatetime(),
            title=row['title'],
            url=row['url']
        )
        news_record_cassandra.save()
    print("Dados das notícias armazenados no Cassandra.")

    # --- Armazenar dados do Fear and Greed Index no Cassandra ---
    print("Armazenando dados do Fear and Greed Index no Cassandra...")
    for index, row in fear_greed_df.iterrows():
        fear_greed_record = FearGreedIndex(
            index_timestamp=row['timestamp'].to_pydatetime(),
            value=row['value'],
            status=row['status']
        )
        fear_greed_record.save()
    print("Dados do Fear and Greed Index armazenados no Cassandra.")

    # --- Armazenar cotação no Cassandra ---
    print("Armazenando Cotação na base Cassandra...")
    for index, row in btc_prices_df.iterrows():
        btc_price_record = BTCPrices(
            partition_key='btc_data',  # Use a constant partition key
            price_timestamp=row['timestamp'].to_pydatetime(),
            price=row['price'],
            percent_change_1h=row['percent_change_1h'],
            percent_change_24h=row['percent_change_24h'],
            percent_change_30d=row['percent_change_30d']
        )
        btc_price_record.save()
    print("Cotação armazenada em Cassandra.")

except Exception as e:
    print(f"Erro em Cassandra: {e}")

finally:
    if cluster:
        cluster.shutdown()
        print("Conexão Cassandra finalizada.")

,timestamp,price,percent_change_1h,percent_change_24h,percent_change_30d
0,2025-04-19 18:39:14.999000+00:00,85265.407158,0.147734,0.812867,1.296796
1,2025-04-19 18:39:36.287000+00:00,85265.407158,0.147734,0.812867,1.296796
2,2025-04-19 18:42:35.553000+00:00,85258.245749,0.229279,0.795166,1.394847
3,2025-04-19 21:47:29.379000+00:00,85282.642370,0.040224,0.772037,0.983622
4,2025-04-20 15:54:16.699000+00:00,84324.978986,-0.297098,-1.007953,0.910979


,timestamp,value,status
0,2025-04-19,32,Fear
1,2025-04-19,32,Fear
2,2025-04-19,32,Fear
3,2025-04-19,32,Fear
4,2025-04-20,37,Fear


,published_at,title,url
0,2025-04-19 14:55:42,What Next For Crypto Market As Trump Signals P...,https://cryptopanic.com/news/21174885/What-Nex...
1,2025-04-19 14:55:42,What Next For Crypto Market As Trump Signals P...,https://cryptopanic.com/news/21174885/What-Nex...
2,2025-04-19 14:55:42,What Next For Crypto Market As Trump Signals P...,https://cryptopanic.com/news/21174885/What-Nex...
3,2025-04-19 15:05:08,Charles Schwab to Launch Bitcoin and Crypto Sp...,https://cryptopanic.com/news/21175163/Charles-...
4,2025-04-19 15:05:08,Charles Schwab to Launch Bitcoin and Crypto Sp...,https://cryptopanic.com/news/21175163/Charles-...


Conectado Cassandra!
Sucesso para executar consulta: 53904aa0-1e44-11f0-a962-5f44ec93de12
Keyspace 'crypto_keyspace' criado ou já existe.
Keyspace 'crypto_keyspace' definido.
Tabela Cassandra Notícias 'news' sincronizada.
Tabela Cassandra Índice de Medo e Ganância 'fear_greed_index' sincronizada.
Tabela Cassandra Preços BTC 'btc_prices' sincronizada.
Armazenando dados das notícias no Cassandra...


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/cassandra/cqlengine/management.py:544: UserWarning: CQLENG_ALLOW_SCHEMA_MANAGEMENT environment variable is not set. Future versions of this package will require this variable to enable management functions.
  warnings.warn(msg)


Dados das notícias armazenados no Cassandra.
Armazenando dados do Fear and Greed Index no Cassandra...
Dados do Fear and Greed Index armazenados no Cassandra.
Armazenando Cotação na base Cassandra...
Cotação armazenada em Cassandra.
Conexão Cassandra finalizada.


<h2> Criar rotina para enviar notificação caso Índice de Medo e Ganância seja maior que 50 <h2>

In [22]:

# --- Configurações de Conexão com Cassandra ---
CASSANDRA_HOSTS = ['127.0.0.1']
KEYSPACE = 'crypto_keyspace'
TABLE_FEAR_GREED = 'fear_greed_index'

# --- Configurações de Email ---
SMTP_SERVER = 'smtp.office365.com'
SMTP_PORT = 587
EMAIL_FROM = '10403308@mackenzista.com.br'
EMAIL_TO = 'diogoluis.campanha@gmail.com'
EMAIL_PASSWORD = 'XXXXXXXX'

# --- Threshold ---
THRESHOLD = 30

# Notificação de Alerta
above_threshold = False

def get_latest_fear_greed_index():
    cluster = None
    try:
        cluster = Cluster(CASSANDRA_HOSTS)
        session = cluster.connect(KEYSPACE)
        query = f"SELECT value FROM {TABLE_FEAR_GREED} LIMIT 1"
        row = session.execute(query).one()
        if row:
            return row.value
        else:
            print("No Fear and Greed Index data found.")
            return None
    except Exception as e:
        print(f"Error querying Cassandra: {e}")
        return None
    finally:
        if cluster:
            cluster.shutdown()

def send_email(subject, body):
    try:
        msg = MIMEText(body)
        msg['Subject'] = subject
        msg['From'] = '10403308@mackenzista.com.br'
        msg['To'] = 'diogoluis.campanha@gmail.com'

        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()  # Use TLS encryption
            server.login(EMAIL_FROM, EMAIL_PASSWORD)
            server.sendmail(EMAIL_FROM, [EMAIL_TO], msg.as_string())
        print(f"Email sent: {subject}")
    except Exception as e:
        print(f"Error sending email: {e}")

# --- Main Monitoring Loop (Limited to 30 seconds using a counter) ---
start_time = time.time()
while time.time() - start_time < 30:
    latest_index = get_latest_fear_greed_index()
    if latest_index is not None:
        current_time = datetime.now()
        current_time_str = current_time.strftime("%Y-%m-%d %H:%M:%S")
        print(f"Current Fear and Greed Index at {current_time_str}: {latest_index}")

        if latest_index > THRESHOLD and not above_threshold:
            subject = "Fear and Greed Index Above Threshold (50)"
            body = f"The Bitcoin Fear and Greed Index has risen above 50. Current value: {latest_index} at {current_time_str}."
            send_email(subject, body)
            above_threshold = True  # Set flag to prevent repeated emails

        elif latest_index <= THRESHOLD:
            above_threshold = False  # Reset flag when below threshold

    time.sleep(5)  # Check every 5 seconds

print("Monitoring stopped after 30 seconds.")

Current Fear and Greed Index at 2025-04-20 21:07:02: 37
Email sent: Fear and Greed Index Above Threshold (50)
Current Fear and Greed Index at 2025-04-20 21:07:08: 37
Current Fear and Greed Index at 2025-04-20 21:07:13: 37
Current Fear and Greed Index at 2025-04-20 21:07:18: 37
Current Fear and Greed Index at 2025-04-20 21:07:23: 37
Current Fear and Greed Index at 2025-04-20 21:07:29: 37
Monitoring stopped after 30 seconds.


<h1>Resumo do Projeto<h1>
<h3>- APIs para coletar dados - 1 ponto<h3>
<h3>- Kafka Producer - 1 ponto<h3>
<h3>- Ingestão com Kafka (Producer + Consumer) - 1 ponto<h3>
<h3>- MongoDB para Data Lake (dados não estruturados) - 1 ponto<h3>
<h3>- ELT Pipeline (transformação de dados do mongoDB) - 1 ponto<h3>
<h3>- Cassandra para dados estruturados (Data Warehouse) - 1 ponto<h3>
<h3>- Grafana para visualização - 1 ponto<h3>
<h3>- Alertas de e-mail para certos dados (threshold) - 1 ponto<h3>
<h3> Total = 8 pontos <h3>